In [ ]:
pip install solana

In [ ]:
pip install base58

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install plotly

# Getting start

In [ ]:
import json
import solana
import time
import logging
import threading
import numpy as np
import urllib.request
import json
from urllib.request import Request, urlopen

import base58
import base64
import struct

from solana.rpc.api import Client
explorerULRAdd = "https://explorer.solana.com/address/"
explorerULRTx = "https://explorer.solana.com/tx/"

# Fill in your Chainstack solana endpoint here:

In [ ]:
http_client = Client("")

# Download data from solana

In [ ]:
lastSignature = None
timeStampNow = int(time.time())
count = 0
mintArr = [] #where data is stored
blkTime = timeStampNow
timeRange = 0.5*60*60 #0.5 hours


def getTxDetail(txSignature):
    txDetail = http_client.get_transaction(txSignature)
    if "result" in txDetail: 
        if "meta" in txDetail["result"]:
            if "postTokenBalances" in txDetail["result"]["meta"]:
                if (txDetail["result"]["meta"]["postTokenBalances"]):
                    if len(txDetail["result"]["meta"]["postTokenBalances"]) == 1:
                        if ("mint" in txDetail["result"]["meta"]["postTokenBalances"][0])and("owner" in txDetail["result"]["meta"]["postTokenBalances"][0]):
                            if txDetail["result"]["meta"]["postTokenBalances"][0]["owner"] == "STEPNq2UGeGSzCyGVr2nMQAzf8xuejwqebd84wcksCK":
                                blkTime = txDetail["result"]["blockTime"]
                                mintAddress = txDetail["result"]["meta"]["postTokenBalances"][0]["mint"]
                                metadataIdx = txDetail["result"]["transaction"]["message"]["instructions"][2]["accounts"][0]
                                metaDataAddr = txDetail["result"]["transaction"]["message"]["accountKeys"][metadataIdx]
                                # Print the mint address
                                print(explorerULRAdd+mintAddress)
                                mintArr.append({"blockTime":blkTime,"txAddress":txSignature,"mintAddress":mintAddress, "metaAddress":metaDataAddr })

# Getting all transaction for minting new shoes in the past half hour
if __name__ == "__main__":
    while(True):
        print("round-"+str(count+1))
        print("getting transactions")
        txs = http_client.get_signatures_for_address("STEPNq2UGeGSzCyGVr2nMQAzf8xuejwqebd84wcksCK",limit=200,before=lastSignature)["result"]
        print("processing signatures")
        signatures = np.array([o["signature"] for o in txs])
        
        threads = list()
        for signature in signatures:
            x = threading.Thread(target=getTxDetail, args=(signature,))
            threads.append(x)
            x.start()
        for index, thread in enumerate(threads):
            thread.join()
        if(txs[-1]['blockTime'] < (timeStampNow-timeRange)):
            break
        else:
            count += 1
            lastSignature = txs[-1]["signature"]
        time.sleep(3)

round-1
getting transactions
processing signatures
https://explorer.solana.com/address/P3TsW7wRNqmJ32Dpk54YtfqA2QMMP6dt4SwHuzkUL5S
https://explorer.solana.com/address/P7MTbPCKy4NyD4s2Yt1rHznHCZjzcpEVY1pCscKoXN2
https://explorer.solana.com/address/EGCnoNo2oH5nvXpXnNrZtN4AshpSbrUQxDBxvpA2fsA4
round-2
getting transactions
processing signatures
https://explorer.solana.com/address/GwVNf1XRb3XbB49Jb1pL1ZyTtxJuLGdL1aau6JuUvGeD
https://explorer.solana.com/address/5XPWbFiuxFbqNhjefq4FeJNoQZ9jAmMGAetHTnQWPdYYhttps://explorer.solana.com/address/CiwjcsmzaZd8jbicAkmoYPvbPGgs2EHdWCTtKozDc33k

https://explorer.solana.com/address/GmJ2KmiUiGV8oqBC9mBGEcsja13vLLT8oRtcmKqGRBTF
round-3
getting transactions
processing signatures
https://explorer.solana.com/address/AisHx1N8shrxgULD7oYcfjwT952Eq5ns1a4xDeKfm95D
https://explorer.solana.com/address/4xDJmtrjtkKa35siAu8PSVGNaChxyj5yn4iArXJKYmJX
https://explorer.solana.com/address/GQW2833CWajghKphuCufq6yMjf1bLR567D17UNs67sVBhttps://explorer.solana.com/address/9eKJ

# Decode metaplex token metadata

In [ ]:
def unpack_metadata_account(rawdata):
    data = base64.b64decode(rawdata)
    assert(data[0] == 4)
    i = 1
    source_account = base58.b58encode(bytes(struct.unpack('<' + "B"*32, data[i:i+32])))
    i += 32
    mint_account = base58.b58encode(bytes(struct.unpack('<' + "B"*32, data[i:i+32])))
    i += 32
    name_len = struct.unpack('<I', data[i:i+4])[0]
    i += 4
    name = struct.unpack('<' + "B"*name_len, data[i:i+name_len])
    i += name_len
    symbol_len = struct.unpack('<I', data[i:i+4])[0]
    i += 4 
    symbol = struct.unpack('<' + "B"*symbol_len, data[i:i+symbol_len])
    i += symbol_len
    uri_len = struct.unpack('<I', data[i:i+4])[0]
    i += 4 
    uri = struct.unpack('<' + "B"*uri_len, data[i:i+uri_len])
    i += uri_len
    fee = struct.unpack('<h', data[i:i+2])[0]
    i += 2
    has_creator = data[i] 
    i += 1
    creators = []
    verified = []
    share = []
    if has_creator:
        creator_len = struct.unpack('<I', data[i:i+4])[0]
        i += 4
        for _ in range(creator_len):
            creator = base58.b58encode(bytes(struct.unpack('<' + "B"*32, data[i:i+32])))
            creators.append(creator)
            i += 32
            verified.append(data[i])
            i += 1
            share.append(data[i])
            i += 1
    primary_sale_happened = bool(data[i])
    i += 1
    is_mutable = bool(data[i])
    metadata = {
        "update_authority": source_account,
        "mint": mint_account,
        "data": {
            "name": bytes(name).decode("utf-8").strip("\x00"),
            "symbol": bytes(symbol).decode("utf-8").strip("\x00"),
            "uri": bytes(uri).decode("utf-8").strip("\x00"),
            "seller_fee_basis_points": fee,
            "creators": creators,
            "verified": verified,
            "share": share,
        },
        "primary_sale_happened": primary_sale_happened,
        "is_mutable": is_mutable,
    }
    return metadata

# Getting the metadata and shoe attributes

In [ ]:
shoeArr = []
sleepTimeBetweenRequest = 1 # 1 second

def getShoeInfo(mintObj):
    data = http_client.get_account_info(mintObj["metaAddress"])
    rawdata = data["result"]["value"]["data"][0]
    metadata = unpack_metadata_account(rawdata)
    mintObj["is_mutable"] = metadata["is_mutable"]
    mintObj["primary_sale_happened"] = metadata["primary_sale_happened"]
    mintObj["symbol"] = metadata["data"]["symbol"]
    mintObj["uri"] = metadata["data"]["uri"]
    mintObj["seller_fee_basis_points"] = metadata["data"]["seller_fee_basis_points"]
    mintObj["creators"] = metadata["data"]["creators"]
    stepnUri = metadata["data"]["uri"]
    req = Request(stepnUri, headers={'User-Agent': 'Mozilla/5.0'})
    shoeData = json.loads(urlopen(req).read())
    mintObj["name"] = shoeData["name"]
    mintObj["description"] = shoeData["description"]
    mintObj["image"] = shoeData["image"]
    for attr in shoeData["attributes"]:
        mintObj[attr["trait_type"]] = attr["value"]
    print(stepnUri)
    shoeArr.append(mintObj)
    

if __name__ == "__main__":
    threads = list()
    for mintObj in mintArr:
        x = threading.Thread(target=getShoeInfo, args=(mintObj,))
        threads.append(x)
        x.start()
        time.sleep(sleepTimeBetweenRequest)
    for index, thread in enumerate(threads):
        thread.join()

https://api.stepn.com/run/nftjson/103/100098400115
https://api.stepn.com/run/nftjson/103/74943160245
https://api.stepn.com/run/nftjson/103/90178694514
https://api.stepn.com/run/nftjson/103/93002654859
https://api.stepn.com/run/nftjson/103/121108538635
https://api.stepn.com/run/nftjson/103/109598897569
https://api.stepn.com/run/nftjson/103/91816402919
https://api.stepn.com/run/nftjson/103/147882251489
https://api.stepn.com/run/nftjson/103/34371383631
https://api.stepn.com/run/nftjson/103/81430058021
https://api.stepn.com/run/nftjson/103/132908497965
https://api.stepn.com/run/nftjson/103/120255620447
https://api.stepn.com/run/nftjson/103/149237521401
https://api.stepn.com/run/nftjson/103/47344326569
https://api.stepn.com/run/nftjson/103/169100562369
https://api.stepn.com/run/nftjson/103/185416094927
https://api.stepn.com/run/nftjson/103/50406358749
https://api.stepn.com/run/nftjson/103/86432283245
https://api.stepn.com/run/nftjson/103/104668000567
https://api.stepn.com/run/nftjson/103/51

# Analysis:

In [ ]:
import pandas as pd

shoedf = pd.DataFrame(shoeArr, columns = shoeArr[0].keys())

### Save to csv

In [ ]:
shoedf.to_csv("save.csv", sep=',', encoding='utf-8')

### Read from csv

In [ ]:
shoedf = pd.read_csv('save.csv')


### Listing all new shoes

In [ ]:
from IPython.display import Image, HTML, display

htmlStr = '<div class="row">'
for shoe in shoeArr:
  try:
    htmlStr += '<img src=' + shoe["image"]+' style="width:150px"> </img>'
    htmlStr += '<p>' + shoe["name"]+'</p>'
    htmlStr += '<p>' + shoe["Sneaker quality"]+'</p>'
    htmlStr += '<p>' + shoe["Sneaker type"]+'</p>'
    mintTime = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(shoe["blockTime"]))
    htmlStr += '<p> Minted at ' + mintTime +'</p>'
    htmlStr += '<a href="'+ explorerULRAdd + shoe["mintAddress"]+'" target=”_blank”>Mint address</a>'
    htmlStr += '<p>***</p>'
  except Exception:
    pass
htmlStr += '<div>'
HTML(htmlStr)

# Shoe types

In [ ]:
import plotly.express as px

fig = px.pie(shoedf, names='Sneaker type', title='Type of shoes')
fig.show()

### Shoe rarity

In [ ]:
import plotly.express as px

fig = px.pie(shoedf, names='Sneaker quality', title='Rarity of shoes')
fig.show()

### Shoe minting count


In [ ]:
import plotly.express as px

fig = px.pie(shoedf, names='Shoe-minting Count', title='Minting count')
fig.show()

### Shoe level

In [ ]:
import plotly.express as px
fig = px.histogram(shoedf, x='Level', title='Level',nbins=20)
fig.show()

### 4 main attributes

In [ ]:
import plotly.express as px
fig = px.histogram(shoedf, x='Efficiency', title='Efficiency',nbins=50,color_discrete_sequence=['indianred'])
fig.show()

In [ ]:
import plotly.express as px
fig = px.histogram(shoedf, x='Luck', title='Luck',nbins=50,color_discrete_sequence=['goldenrod'])
fig.show()

In [ ]:
import plotly.express as px
fig = px.histogram(shoedf, x='Comfortability', title='Comfortability',nbins=50,color_discrete_sequence=['blue'])
fig.show()

In [ ]:
import plotly.express as px
fig = px.histogram(shoedf, x='Resilience', title='Resilience',nbins=50,color_discrete_sequence=['magenta'])
fig.show()

### Socket

In [ ]:
import plotly.express as px

tempdf = shoedf["Socket 1"]
tempdf = tempdf.append(shoedf["Socket 2"])
tempdf = tempdf.append(shoedf["Socket 3"])
tempdf = tempdf.append(shoedf["Socket 4"])
tempdf = pd.DataFrame(tempdf.to_list(), columns = ["socket"])

fig = px.pie(tempdf, names='socket', title='Minting count')
fig.show()